In [1]:
import pandas as pd
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import progressbar
from time import sleep
import re

os.chdir('newdir')

In [2]:
source_data = pd.read_csv('data.csv',encoding='ISO-8859-1')
match_set = pd.read_csv('match_data.csv',encoding='ISO-8859-1')
states =  pd.read_csv('states.csv',encoding='ISO-8859-1')

source_data = source_data.merge(states,left_on='Location Address State/Province',right_on='Abbrev',how='left')
match_set = match_set.merge(states,left_on='stateprovcode',right_on='Abbrev',how='left')
source_data['State'] = source_data['State'].combine_first(source_data['Location Address State/Province'])
match_set['matchname'] = match_set['Location Name'] + ' ' + match_set['Location Address City']
source_data['matchname'] = source_data['locationname'] + ' ' + source_data['city']
match_set = match_set.dropna(subset=['matchname'])

In [3]:
source_data['matchname'].replace(to_replace='[^0-9a-zA-Z ]+', value='',regex=True, inplace=True) 
source_data['State'].replace(to_replace='[^0-9a-zA-Z ]+', value=' ',regex=True, inplace=True)
match_set['matchname'].replace(to_replace='[^0-9a-zA-Z ]+', value='',regex=True, inplace=True) 
match_set['State'].replace(to_replace='[^0-9a-zA-Z ]+', value=' ',regex=True, inplace=True)

In [4]:
source_data = source_data[source_data['State'].str.lower().isin(match_set['State'].str.lower())]
source_data = source_data.dropna(subset=['matchname','State'])

In [5]:
matchy = pd.DataFrame(columns=['IDENTIFIER','match','score'])

bar = progressbar.ProgressBar(maxval=len(fred_mb), \
    widgets=[progressbar.Bar('☮', '[', ']'), ' ', progressbar.Percentage()])
bar.start()

def fuzzy_match(lookup_value):
    e = 0
    for i in range(len(lookup_value)):
        match_suggested, score = process.extractOne(lookup_value.iloc[i],match_set[mb_match['State'].str.lower() == source_data.iloc[i]['State'].lower()]['matchname'].tolist(),scorer=fuzz.token_set_ratio)
        if lookup_value.iloc[i] in pd.unique(match_set['matchname']).tolist():
            matchy.loc[-1] = [lookup_value.iloc[i], lookup_value.iloc[i], 100]
            matchy.index = matchy.index + 1
            matchy.sort_index(inplace=True)
        else:
            matchy.loc[-1] = [lookup_value.iloc[i], match_suggested, score]
            matchy.index = matchy.index + 1
            matchy.sort_index(inplace=True)
        e+=1
        bar.update(e)        
    return matchy

matching = fuzzy_match(source_data['matchname'])

bar.finish()

[☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮☮] 100%


In [21]:
fuzzed_df = source_data.merge(matchy,left_on='matchname',right_on='IDENTIFIER',how='left')
fuzzed_df = fuzzed_df.merge(mb_match,left_on='match',right_on='matchname',how='left')
fuzzed_df.to_csv('fuzzy_matched_output.csv')

In [23]:
print(str("{:.0%}".format(1-(len(fuzzed_df[(fuzzed_df['Data Source External ID'].isna()) &\
                                           (fuzzed_df['score']<88)])/len(fuzzed_df)))) + \
      ' quality matches aka ' + str(len(fuzzed_df)-len(fuzzed_df[(fuzzed_df['Data Source External ID'].isna()) &\
                                                  (fuzzed_df['score']<88)]))+ \
      ' out of '+str(len(fuzzed_df)))

81% quality matches aka 1099 out of 1351


In [8]:
!jupyter nbconvert --to script fuzzy-wuzzy.ipynb

[NbConvertApp] Converting notebook fuzzy-freddy.ipynb to script
[NbConvertApp] Writing 3600 bytes to fuzzy-freddy.py
